In [14]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('plate_details.csv', delimiter=',')
# print(df)

# Exclude rows where the filter column is empty
df = df[df['filter'].notna()]
# print(df)

# Group the DataFrame by observatory and save each group to a separate CSV file
for observatory, group_df in df.groupby('observatory'):
    filename = f"{observatory.strip().replace(' ', '_')}.csv"
    group_df.to_csv(filename, index=False)
    print(f"Saved group '{observatory}' to '{filename}'.")

Saved group 'Astrophysikalische Observatorium Potsdam' to 'Astrophysikalische_Observatorium_Potsdam.csv'.
Saved group 'Dr. Remeis-Observatory' to 'Dr._Remeis-Observatory.csv'.
Saved group 'Dr. Remeis-Observatory, Southern Station' to 'Dr._Remeis-Observatory,_Southern_Station.csv'.
Saved group 'European Southern Observatory' to 'European_Southern_Observatory.csv'.
Saved group 'Hamburger Sternwarte' to 'Hamburger_Sternwarte.csv'.
Saved group 'Thüringer Landesternwarte Tautenburg' to 'Thüringer_Landesternwarte_Tautenburg.csv'.
Saved group 'Vatican Observatory' to 'Vatican_Observatory.csv'.


In [45]:
import requests
import pyvo as vo
import json
import numpy as np
import pandas as pd

# Define the parameters
name = 'APPLAUSE'
url = 'https://www.plate-archive.org/tap'
token = 'Token 863d6c56ca291ffc99090f6b2cefb35f496ca8e4'

# Read the CSV file and select the column with the plate_id
df = pd.read_csv('Thüringer_Landesternwarte_Tautenburg.csv')
plate_ids = df['plate_id'].tolist()

# Function to chunk the list into groups of specified size
def chunk_list(data_list, chunk_size):
    for i in range(0, len(data_list), chunk_size):
        yield data_list[i:i + chunk_size]

# Function to process each chunk
def process_chunk(chunk):
    # Format the plate_id list for the SQL query
    ids_str = ', '.join(f"'{id}'" for id in chunk)
    
    # Create the query string
    qstr = f"""
    SELECT exposure_id, plate_id, ra_orig, dec_orig, jd_start, exptime
    FROM applause_dr4.exposure
    WHERE plate_id IN ({ids_str})
    """
    
    # Setup the TAP service session
    tap_session = requests.Session()
    tap_session.headers['Authorization'] = token
    tap_service = vo.dal.TAPService(url, session=tap_session)
    
    # Submit the query
    lang = 'PostgreSQL'
    job = tap_service.submit_job(qstr, language=lang)
    job.run()
    
    # Wait for job completion
    job.wait(phases=["COMPLETED", "ERROR", "ABORTED"], timeout=30.)
    
    # Raise an error if the job failed
    job.raise_if_error()
    
    # Fetch results
    return job.fetch_result()

# Print pyvo version and TAP service name
print('\npyvo version %s \n' % vo.__version__)
print('TAP service %s \n' % name)

# Initialize the result list
query_results = []

# Process each chunk
for chunk in chunk_list(plate_ids, 170):
    results = process_chunk(chunk)
    query_results.append(results)

# Print the final query results as DataFrame
for result in query_results:
    df_result = result.to_table().to_pandas()
    print(df_result)


pyvo version 1.5.2 

TAP service APPLAUSE 

     exposure_id  plate_id   ra_orig dec_orig      jd_start  exptime
0         115572     70372     00:40   +41:00  2.437255e+06    240.0
1         115573     70373     00:40   +41:00  2.437255e+06    600.0
2         115574     70374     03:44   +24:00  2.437256e+06    600.0
3         115575     70375     00:40   +41:00  2.437261e+06   2340.0
4         115576     70376  03:45:05   +23:59  2.437263e+06   1740.0
..           ...       ...       ...      ...           ...      ...
167       115739     70537  01:39:46   +51:19  2.437913e+06   1080.0
168       115740     70538  03:06:22   +61:20  2.437913e+06    240.0
169       115741     70539  16:43:00   +36:33  2.437913e+06   3600.0
170       115742     70540     18:50   +10:16  2.437913e+06    600.0
171       115743     70541     18:50   +10:16  2.437913e+06    900.0

[172 rows x 6 columns]
     exposure_id  plate_id   ra_orig dec_orig      jd_start  exptime
0         115744     70542     18:

In [47]:
# Initialize an empty DataFrame to store the combined results
combined_df = pd.DataFrame()

# Combine the query results into a single DataFrame
for result in query_results:
    df_result = result.to_table().to_pandas()
    combined_df = pd.concat([combined_df, df_result], ignore_index=True)

# Save the combined DataFrame as a CSV file named 'plate_tls.csv'
combined_df.to_csv('plate_tls.csv', index=False)

In [53]:
import pandas as pd

# Read the CSV files into DataFrames
df1 = pd.read_csv('plate_tls.csv')
df2 = pd.read_csv('Thüringer_Landesternwarte_Tautenburg.csv')

# Merge the DataFrames on 'plate_id'
merged_df = pd.merge(df2, df1[['plate_id', 'ra_orig', 'dec_orig']], on='plate_id')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('TLS_plate_pos.csv', index=False)

print("Merged file saved as 'TLS_plate_pos.csv'")

Merged file saved as 'TLS_plate_pos.csv'


In [6]:
import pandas as pd
from datetime import datetime, timedelta

# Function to filter plates taken within a one-year period and have different filters
def filter_plates_within_year(df):
    filtered_plates = []
    for i in range(len(df)):
        for j in range(i+1, len(df)):
            date_i = datetime.strptime(df.iloc[i]['date_orig'], '%Y-%m-%d')
            date_j = datetime.strptime(df.iloc[j]['date_orig'], '%Y-%m-%d')
            delta = date_j - date_i
            if abs(delta.days) <= 124 and df.iloc[i]['filter'] != df.iloc[j]['filter']:
                filtered_plates.append(df.iloc[i])
                filtered_plates.append(df.iloc[j])
    return pd.DataFrame(filtered_plates)

# Read the CSV file
filename = "TLS_plate_pos.csv"
# filename = "Vatican_Observatory.csv"
# filename = "Dr._Remeis-Observatory.csv"
df = pd.read_csv(filename)

# Sort by date_orig
df_sorted = df.sort_values(by='date_orig')

# Filter plates within one-year period with different filters
filtered_df = filter_plates_within_year(df_sorted)

# Save filtered DataFrame to CSV
filtered_df.to_csv("TLS.csv", index=False)

In [7]:
print(df[80:85])

    plate_id  plate_num                                        emulsion  \
80     70450        299  yellow-sensitive plate (D; G; ZP1; ZO; AP; RO)   
81     70451        300  yellow-sensitive plate (D; G; ZP1; ZO; AP; RO)   
82     70452        301  yellow-sensitive plate (D; G; ZP1; ZO; AP; RO)   
83     70453        302  yellow-sensitive plate (D; G; ZP1; ZO; AP; RO)   
84     70454        303  yellow-sensitive plate (D; G; ZP1; ZO; AP; RO)   

            filter   date_orig                           observatory  \
80  without filter  1962-02-09  Thüringer Landesternwarte Tautenburg   
81  without filter  1962-02-09  Thüringer Landesternwarte Tautenburg   
82  without filter  1962-02-10  Thüringer Landesternwarte Tautenburg   
83  without filter  1962-02-10  Thüringer Landesternwarte Tautenburg   
84  without filter  1962-02-10  Thüringer Landesternwarte Tautenburg   

     ra_orig dec_orig  
80  05:30:50   +34:40  
81  05:52:42   +32:05  
82  05:54:52   +49:53  
83  08:36:40   +20:0

In [13]:
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy.time import Time
import astropy.units as u

# Load the CSV file into a DataFrame
file_path = "TLS.csv"  # Replace with your actual file path
df = pd.read_csv(file_path)

# Function to convert ra_orig and dec_orig to Galactic coordinates with date
def convert_to_galactic_with_date(ra, dec, date_orig):
    time = Time(date_orig)
    skycoord = SkyCoord(ra=ra, dec=dec, unit=(u.hourangle, u.deg), obstime=time)
    return skycoord.galactic

# Apply the conversion function to ra_orig, dec_orig, and date_orig
df['galactic_coord'] = df.apply(lambda row: convert_to_galactic_with_date(row['ra_orig'], row['dec_orig'], row['date_orig']), axis=1)

# Define a tolerance (in degrees)
tolerance = 30 * u.arcminute  # Adjust as needed

# Initialize an empty list to store the matched rows
matched_rows = []

# Iterate through the DataFrame in pairs
for i in range(0, len(df), 2):
    if i + 1 < len(df):
        # Compare the pairs
        row1 = df.iloc[i]
        row2 = df.iloc[i + 1]
        
        # Calculate the separation between the two galactic coordinates
        separation = row1['galactic_coord'].separation(row2['galactic_coord'])
        
        if separation <= tolerance:
            matched_rows.append(row1)
            matched_rows.append(row2)

# Create a new DataFrame from the matched rows
matched_df = pd.DataFrame(matched_rows).drop(columns=['galactic_coord'])

# Save the resulting DataFrame to a new CSV file
output_file_path = "matched_rows_galactic_TLS.csv"
matched_df.to_csv(output_file_path, index=False)

print(f"Saved matched rows to '{output_file_path}'")

Saved matched rows to 'matched_rows_galactic_TLS.csv'


In [14]:
import requests
import pyvo as vo
import json
import numpy as np
import pandas as pd

# Define the parameters
name = 'APPLAUSE'
url = 'https://www.plate-archive.org/tap'
token = 'Token 863d6c56ca291ffc99090f6b2cefb35f496ca8e4'

# Read the CSV file and select the column with the plate_id
df = pd.read_csv('Hamburger_Sternwarte.csv')
plate_ids = df['plate_id'].tolist()

# Function to chunk the list into groups of specified size
def chunk_list(data_list, chunk_size):
    for i in range(0, len(data_list), chunk_size):
        yield data_list[i:i + chunk_size]

# Function to process each chunk
def process_chunk(chunk):
    # Format the plate_id list for the SQL query
    ids_str = ', '.join(f"'{id}'" for id in chunk)
    
    # Create the query string
    qstr = f"""
    SELECT exposure_id, plate_id, ra_orig, dec_orig, jd_start, exptime
    FROM applause_dr4.exposure
    WHERE plate_id IN ({ids_str})
    """
    
    # Setup the TAP service session
    tap_session = requests.Session()
    tap_session.headers['Authorization'] = token
    tap_service = vo.dal.TAPService(url, session=tap_session)
    
    # Submit the query
    lang = 'PostgreSQL'
    job = tap_service.submit_job(qstr, language=lang)
    job.run()
    
    # Wait for job completion
    job.wait(phases=["COMPLETED", "ERROR", "ABORTED"], timeout=30.)
    
    # Raise an error if the job failed
    job.raise_if_error()
    
    # Fetch results
    return job.fetch_result()

# Print pyvo version and TAP service name
print('\npyvo version %s \n' % vo.__version__)
print('TAP service %s \n' % name)

# Initialize the result list
query_results = []

# Process each chunk
for chunk in chunk_list(plate_ids, 170):
    results = process_chunk(chunk)
    query_results.append(results)

# Print the final query results as DataFrame
for result in query_results:
    df_result = result.to_table().to_pandas()
    print(df_result)


pyvo version 1.5.2 

TAP service APPLAUSE 

     exposure_id  plate_id  ra_orig  dec_orig      jd_start  exptime
0           2628      1865  02:38.0  +50:17.7  2.425174e+06  14400.0
1           9157      5863  23:00.0  +83 00.0  2.431289e+06   3180.0
2           9417      5935  04:24.0  +15:48.0  2.431798e+06   2460.0
3           9418      5935  04:24.0  +15:48.0           NaN    420.0
4           9419      5935  04:24.0  +15:48.0           NaN    240.0
..           ...       ...      ...       ...           ...      ...
313        14540      7979  05:43.3  +09:02.0  2.435129e+06    180.0
314        14550      7988  05:43.3  +09:02.0  2.435131e+06    360.0
315        14551      7988  05:43.3  +09:02.0  2.435131e+06    180.0
316        14558      7994  05:43.3  +09:02.0  2.435137e+06    360.0
317        14559      7994  05:43.3  +09:02.0  2.435137e+06    180.0

[318 rows x 6 columns]
     exposure_id  plate_id  ra_orig  dec_orig      jd_start  exptime
0          14591      8024  05:43.

In [15]:
# Initialize an empty DataFrame to store the combined results
combined_df = pd.DataFrame()

# Combine the query results into a single DataFrame
for result in query_results:
    df_result = result.to_table().to_pandas()
    combined_df = pd.concat([combined_df, df_result], ignore_index=True)

# Save the combined DataFrame as a CSV file named 'plate_tls.csv'
combined_df.to_csv('plate_hs.csv', index=False)

In [16]:
import pandas as pd

# Read the CSV files into DataFrames
df1 = pd.read_csv('plate_hs.csv')
df2 = pd.read_csv('Hamburger_Sternwarte.csv')

# Merge the DataFrames on 'plate_id'
merged_df = pd.merge(df2, df1[['plate_id', 'ra_orig', 'dec_orig']], on='plate_id')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('HS_plate_pos.csv', index=False)

print("Merged file saved as 'HS_plate_pos.csv'")

Merged file saved as 'HS_plate_pos.csv'


In [18]:
import pandas as pd
from datetime import datetime, timedelta

# Function to filter plates taken within a one-year period and have different filters
def filter_plates_within_year(df):
    filtered_plates = []
    for i in range(len(df)):
        for j in range(i+1, len(df)):
            date_i = datetime.strptime(df.iloc[i]['date_orig'], '%Y-%m-%d')
            date_j = datetime.strptime(df.iloc[j]['date_orig'], '%Y-%m-%d')
            delta = date_j - date_i
            if abs(delta.days) <= 124 and df.iloc[i]['filter'] != df.iloc[j]['filter']:
                filtered_plates.append(df.iloc[i])
                filtered_plates.append(df.iloc[j])
    return pd.DataFrame(filtered_plates)

# Read the CSV file
filename = "HS_plate_pos.csv"
# filename = "TLS_plate_pos.csv"
# filename = "Vatican_Observatory.csv"
# filename = "Dr._Remeis-Observatory.csv"
df = pd.read_csv(filename)

# Sort by date_orig
df_sorted = df.sort_values(by='date_orig')

# Filter plates within one-year period with different filters
filtered_df = filter_plates_within_year(df_sorted)

# Save filtered DataFrame to CSV
filtered_df.to_csv("HS.csv", index=False)

In [2]:
import requests
import pyvo as vo
import json
import numpy as np
import pandas as pd

# Define the parameters
name = 'APPLAUSE'
url = 'https://www.plate-archive.org/tap'
token = 'Token 863d6c56ca291ffc99090f6b2cefb35f496ca8e4'

# Read the CSV file and select the column with the plate_id
df = pd.read_csv('Astrophysikalische_Observatorium_Potsdam.csv')
plate_ids = df['plate_id'].tolist()

# Function to chunk the list into groups of specified size
def chunk_list(data_list, chunk_size):
    for i in range(0, len(data_list), chunk_size):
        yield data_list[i:i + chunk_size]

# Function to process each chunk
def process_chunk(chunk):
    # Format the plate_id list for the SQL query
    ids_str = ', '.join(f"'{id}'" for id in chunk)
    
    # Create the query string
    qstr = f"""
    SELECT exposure_id, plate_id, ra_orig, dec_orig, jd_start, exptime
    FROM applause_dr4.exposure
    WHERE plate_id IN ({ids_str})
    """
    
    # Setup the TAP service session
    tap_session = requests.Session()
    tap_session.headers['Authorization'] = token
    tap_service = vo.dal.TAPService(url, session=tap_session)
    
    # Submit the query
    lang = 'PostgreSQL'
    job = tap_service.submit_job(qstr, language=lang)
    job.run()
    
    # Wait for job completion
    job.wait(phases=["COMPLETED", "ERROR", "ABORTED"], timeout=30.)
    
    # Raise an error if the job failed
    job.raise_if_error()
    
    # Fetch results
    return job.fetch_result()

# Print pyvo version and TAP service name
print('\npyvo version %s \n' % vo.__version__)
print('TAP service %s \n' % name)

# Initialize the result list
query_results = []

# Process each chunk
for chunk in chunk_list(plate_ids, 170):
    results = process_chunk(chunk)
    query_results.append(results)

# Print the final query results as DataFrame
for result in query_results:
    df_result = result.to_table().to_pandas()
    print(df_result)


pyvo version 1.5.2 

TAP service APPLAUSE 

     exposure_id  plate_id ra_orig dec_orig      jd_start  exptime
0          77676     24193                   2.427449e+06   1200.0
1          77677     24193                   2.427449e+06    600.0
2          77678     24194                   2.427449e+06   3600.0
3          77679     24195                   2.427449e+06    600.0
4          77680     24195                   2.427449e+06    240.0
..           ...       ...     ...      ...           ...      ...
364        78049     24360                   2.427721e+06   7200.0
365        78050     24361                   2.427733e+06   1800.0
366        78051     24362                   2.427733e+06    360.0
367        78052     24363                   2.427733e+06   2100.0
368        78053     24364                   2.427733e+06    180.0

[369 rows x 6 columns]
     exposure_id  plate_id ra_orig dec_orig      jd_start  exptime
0          78054     24365                   2.427733e+06   

In [3]:
# Initialize an empty DataFrame to store the combined results
combined_df = pd.DataFrame()

# Combine the query results into a single DataFrame
for result in query_results:
    df_result = result.to_table().to_pandas()
    combined_df = pd.concat([combined_df, df_result], ignore_index=True)

# Save the combined DataFrame as a CSV file named 'plate_tls.csv'
combined_df.to_csv('plate_aop.csv', index=False)

In [4]:
import pandas as pd

# Read the CSV files into DataFrames
df1 = pd.read_csv('plate_aop.csv')
df2 = pd.read_csv('Astrophysikalische_Observatorium_Potsdam.csv')

# Merge the DataFrames on 'plate_id'
merged_df = pd.merge(df2, df1[['plate_id', 'ra_orig', 'dec_orig']], on='plate_id')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('AOP_plate_pos.csv', index=False)

print("Merged file saved as 'AOP_plate_pos.csv'")

Merged file saved as 'AOP_plate_pos.csv'


In [5]:
import pandas as pd
from datetime import datetime, timedelta

# Function to filter plates taken within a one-year period and have different filters
def filter_plates_within_year(df):
    filtered_plates = []
    for i in range(len(df)):
        for j in range(i+1, len(df)):
            date_i = datetime.strptime(df.iloc[i]['date_orig'], '%Y-%m-%d')
            date_j = datetime.strptime(df.iloc[j]['date_orig'], '%Y-%m-%d')
            delta = date_j - date_i
            if abs(delta.days) <= 124 and df.iloc[i]['filter'] != df.iloc[j]['filter']:
                filtered_plates.append(df.iloc[i])
                filtered_plates.append(df.iloc[j])
    return pd.DataFrame(filtered_plates)

# Read the CSV file
filename = "AOP_plate_pos.csv"
# filename = "HS_plate_pos.csv"
# filename = "TLS_plate_pos.csv"
# filename = "Vatican_Observatory.csv"
# filename = "Dr._Remeis-Observatory.csv"
df = pd.read_csv(filename)

# Sort by date_orig
df_sorted = df.sort_values(by='date_orig')

# Filter plates within one-year period with different filters
filtered_df = filter_plates_within_year(df_sorted)

# Save filtered DataFrame to CSV
filtered_df.to_csv("AOP.csv", index=False)

In [7]:
import requests
import pyvo as vo
import json
import numpy as np
import pandas as pd

# Define the parameters
name = 'APPLAUSE'
url = 'https://www.plate-archive.org/tap'
token = 'Token 863d6c56ca291ffc99090f6b2cefb35f496ca8e4'

# Read the CSV file and select the column with the plate_id
df = pd.read_csv('European_Southern_Observatory.csv')
plate_ids = df['plate_id'].tolist()

# Function to chunk the list into groups of specified size
def chunk_list(data_list, chunk_size):
    for i in range(0, len(data_list), chunk_size):
        yield data_list[i:i + chunk_size]

# Function to process each chunk
def process_chunk(chunk):
    # Format the plate_id list for the SQL query
    ids_str = ', '.join(f"'{id}'" for id in chunk)
    
    # Create the query string
    qstr = f"""
    SELECT exposure_id, plate_id, ra_orig, dec_orig, jd_start, exptime
    FROM applause_dr4.exposure
    WHERE plate_id IN ({ids_str})
    """
    
    # Setup the TAP service session
    tap_session = requests.Session()
    tap_session.headers['Authorization'] = token
    tap_service = vo.dal.TAPService(url, session=tap_session)
    
    # Submit the query
    lang = 'PostgreSQL'
    job = tap_service.submit_job(qstr, language=lang)
    job.run()
    
    # Wait for job completion
    job.wait(phases=["COMPLETED", "ERROR", "ABORTED"], timeout=30.)
    
    # Raise an error if the job failed
    job.raise_if_error()
    
    # Fetch results
    return job.fetch_result()

# Print pyvo version and TAP service name
print('\npyvo version %s \n' % vo.__version__)
print('TAP service %s \n' % name)

# Initialize the result list
query_results = []

# Process each chunk
for chunk in chunk_list(plate_ids, 170):
    results = process_chunk(chunk)
    query_results.append(results)

# Print the final query results as DataFrame
for result in query_results:
    df_result = result.to_table().to_pandas()
    print(df_result)


pyvo version 1.5.2 

TAP service APPLAUSE 

     exposure_id  plate_id   ra_orig   dec_orig      jd_start  exptime
0          95168     63651  17:37:00  -35:00:00  2.443005e+06   3600.0
1          95169     63652  17:37:00  -35:00:00  2.443006e+06   1200.0
2          95216     63653  17:39:40  -32:30:40  2.443036e+06   3600.0
3          95217     63654  17:39:40  -32:30:40  2.443037e+06   1200.0
4          95173     63655  17:36:12  -37:50:36  2.443007e+06   7200.0
..           ...       ...       ...        ...           ...      ...
165        96761     63908  07:27:59  -01:05:58  2.448272e+06   2400.0
166        96762     63909  07:27:59  -01:06:13  2.448273e+06   2400.0
167        96763     63910  08:08:18  -07:52:41  2.448275e+06   2400.0
168        96764     63911  08:08:18  -07:52:41  2.448275e+06   2400.0
169        96765     63912  07:27:58  -01:07:08  2.448276e+06   2400.0

[170 rows x 6 columns]
     exposure_id  plate_id   ra_orig   dec_orig      jd_start  exptime
0       

In [8]:
# Initialize an empty DataFrame to store the combined results
combined_df = pd.DataFrame()

# Combine the query results into a single DataFrame
for result in query_results:
    df_result = result.to_table().to_pandas()
    combined_df = pd.concat([combined_df, df_result], ignore_index=True)

# Save the combined DataFrame as a CSV file named 'plate_tls.csv'
combined_df.to_csv('plate_eso.csv', index=False)

In [9]:
import pandas as pd

# Read the CSV files into DataFrames
df1 = pd.read_csv('plate_eso.csv')
df2 = pd.read_csv('European_Southern_Observatory.csv')

# Merge the DataFrames on 'plate_id'
merged_df = pd.merge(df2, df1[['plate_id', 'ra_orig', 'dec_orig']], on='plate_id')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('ESO_plate_pos.csv', index=False)

print("Merged file saved as 'ESO_plate_pos.csv'")

Merged file saved as 'ESO_plate_pos.csv'


In [10]:
import pandas as pd
from datetime import datetime, timedelta

# Function to filter plates taken within a one-year period and have different filters
def filter_plates_within_year(df):
    filtered_plates = []
    for i in range(len(df)):
        for j in range(i+1, len(df)):
            date_i = datetime.strptime(df.iloc[i]['date_orig'], '%Y-%m-%d')
            date_j = datetime.strptime(df.iloc[j]['date_orig'], '%Y-%m-%d')
            delta = date_j - date_i
            if abs(delta.days) <= 124 and df.iloc[i]['filter'] != df.iloc[j]['filter']:
                filtered_plates.append(df.iloc[i])
                filtered_plates.append(df.iloc[j])
    return pd.DataFrame(filtered_plates)

# Read the CSV file
filename = "ESO_plate_pos.csv"
# filename = "AOP_plate_pos.csv"
# filename = "HS_plate_pos.csv"
# filename = "TLS_plate_pos.csv"
# filename = "Vatican_Observatory.csv"
# filename = "Dr._Remeis-Observatory.csv"
df = pd.read_csv(filename)

# Sort by date_orig
df_sorted = df.sort_values(by='date_orig')

# Filter plates within one-year period with different filters
filtered_df = filter_plates_within_year(df_sorted)

# Save filtered DataFrame to CSV
filtered_df.to_csv("ESO.csv", index=False)

In [12]:
import requests
import pyvo as vo
import json
import numpy as np
import pandas as pd

# Define the parameters
name = 'APPLAUSE'
url = 'https://www.plate-archive.org/tap'
token = 'Token 863d6c56ca291ffc99090f6b2cefb35f496ca8e4'

# Read the CSV file and select the column with the plate_id
df = pd.read_csv('Dr._Remeis-Observatory.csv')
plate_ids = df['plate_id'].tolist()

# Function to chunk the list into groups of specified size
def chunk_list(data_list, chunk_size):
    for i in range(0, len(data_list), chunk_size):
        yield data_list[i:i + chunk_size]

# Function to process each chunk
def process_chunk(chunk):
    # Format the plate_id list for the SQL query
    ids_str = ', '.join(f"'{id}'" for id in chunk)
    
    # Create the query string
    qstr = f"""
    SELECT exposure_id, plate_id, ra_orig, dec_orig, jd_start, exptime
    FROM applause_dr4.exposure
    WHERE plate_id IN ({ids_str})
    """
    
    # Setup the TAP service session
    tap_session = requests.Session()
    tap_session.headers['Authorization'] = token
    tap_service = vo.dal.TAPService(url, session=tap_session)
    
    # Submit the query
    lang = 'PostgreSQL'
    job = tap_service.submit_job(qstr, language=lang)
    job.run()
    
    # Wait for job completion
    job.wait(phases=["COMPLETED", "ERROR", "ABORTED"], timeout=30.)
    
    # Raise an error if the job failed
    job.raise_if_error()
    
    # Fetch results
    return job.fetch_result()

# Print pyvo version and TAP service name
print('\npyvo version %s \n' % vo.__version__)
print('TAP service %s \n' % name)

# Initialize the result list
query_results = []

# Process each chunk
for chunk in chunk_list(plate_ids, 170):
    results = process_chunk(chunk)
    query_results.append(results)

# Print the final query results as DataFrame
for result in query_results:
    df_result = result.to_table().to_pandas()
    print(df_result)


pyvo version 1.5.2 

TAP service APPLAUSE 

    exposure_id  plate_id ra_orig dec_orig      jd_start  exptime
0        119679     77005                   2.434913e+06    600.0
1        119681     77007                   2.434914e+06    600.0
2        119683     77009                   2.434914e+06    900.0
3        119685     77011                   2.434914e+06    600.0
4        119687     77013                   2.434918e+06    600.0
5        119689     77015                   2.434918e+06    420.0
6        119691     77017                   2.434918e+06    600.0
7        119693     77019                   2.434922e+06    600.0
8        119695     77021                   2.434922e+06    900.0
9        119697     77023                   2.434922e+06    600.0
10       119703     77029                   2.434929e+06    600.0
11       119705     77031                   2.434929e+06    600.0
12       119707     77033                   2.434929e+06    300.0


In [13]:
# Initialize an empty DataFrame to store the combined results
combined_df = pd.DataFrame()

# Combine the query results into a single DataFrame
for result in query_results:
    df_result = result.to_table().to_pandas()
    combined_df = pd.concat([combined_df, df_result], ignore_index=True)

# Save the combined DataFrame as a CSV file named 'plate_tls.csv'
combined_df.to_csv('plate_dro.csv', index=False)

In [16]:
import pandas as pd

# Read the CSV files into DataFrames
df1 = pd.read_csv('plate_dro.csv')
df2 = pd.read_csv('Dr._Remeis-Observatory.csv')

# Merge the DataFrames on 'plate_id'
merged_df = pd.merge(df2, df1[['plate_id', 'ra_orig', 'dec_orig']], on='plate_id')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('DRO_plate_pos.csv', index=False)

print("Merged file saved as 'DRO_plate_pos.csv'")

Merged file saved as 'DRO_plate_pos.csv'


In [17]:
import pandas as pd
from datetime import datetime, timedelta

# Function to filter plates taken within a one-year period and have different filters
def filter_plates_within_year(df):
    filtered_plates = []
    for i in range(len(df)):
        for j in range(i+1, len(df)):
            date_i = datetime.strptime(df.iloc[i]['date_orig'], '%Y-%m-%d')
            date_j = datetime.strptime(df.iloc[j]['date_orig'], '%Y-%m-%d')
            delta = date_j - date_i
            if abs(delta.days) <= 124 and df.iloc[i]['filter'] != df.iloc[j]['filter']:
                filtered_plates.append(df.iloc[i])
                filtered_plates.append(df.iloc[j])
    return pd.DataFrame(filtered_plates)

# Read the CSV file
filename = "DRO_plate_pos.csv"
# filename = "ESO_plate_pos.csv"
# filename = "AOP_plate_pos.csv"
# filename = "HS_plate_pos.csv"
# filename = "TLS_plate_pos.csv"
# filename = "Vatican_Observatory.csv"
# filename = "Dr._Remeis-Observatory.csv"
df = pd.read_csv(filename)

# Sort by date_orig
df_sorted = df.sort_values(by='date_orig')

# Filter plates within one-year period with different filters
filtered_df = filter_plates_within_year(df_sorted)

# Save filtered DataFrame to CSV
filtered_df.to_csv("DRO.csv", index=False)

In [19]:
import requests
import pyvo as vo
import json
import numpy as np
import pandas as pd

# Define the parameters
name = 'APPLAUSE'
url = 'https://www.plate-archive.org/tap'
token = 'Token 863d6c56ca291ffc99090f6b2cefb35f496ca8e4'

# Read the CSV file and select the column with the plate_id
df = pd.read_csv('Dr._Remeis-Observatory,_Southern_Station.csv')
plate_ids = df['plate_id'].tolist()

# Function to chunk the list into groups of specified size
def chunk_list(data_list, chunk_size):
    for i in range(0, len(data_list), chunk_size):
        yield data_list[i:i + chunk_size]

# Function to process each chunk
def process_chunk(chunk):
    # Format the plate_id list for the SQL query
    ids_str = ', '.join(f"'{id}'" for id in chunk)
    
    # Create the query string
    qstr = f"""
    SELECT exposure_id, plate_id, ra_orig, dec_orig, jd_start, exptime
    FROM applause_dr4.exposure
    WHERE plate_id IN ({ids_str})
    """
    
    # Setup the TAP service session
    tap_session = requests.Session()
    tap_session.headers['Authorization'] = token
    tap_service = vo.dal.TAPService(url, session=tap_session)
    
    # Submit the query
    lang = 'PostgreSQL'
    job = tap_service.submit_job(qstr, language=lang)
    job.run()
    
    # Wait for job completion
    job.wait(phases=["COMPLETED", "ERROR", "ABORTED"], timeout=30.)
    
    # Raise an error if the job failed
    job.raise_if_error()
    
    # Fetch results
    return job.fetch_result()

# Print pyvo version and TAP service name
print('\npyvo version %s \n' % vo.__version__)
print('TAP service %s \n' % name)

# Initialize the result list
query_results = []

# Process each chunk
for chunk in chunk_list(plate_ids, 170):
    results = process_chunk(chunk)
    query_results.append(results)

# Print the final query results as DataFrame
for result in query_results:
    df_result = result.to_table().to_pandas()
    print(df_result)


pyvo version 1.5.2 

TAP service APPLAUSE 

     exposure_id  plate_id   ra_orig   dec_orig      jd_start  exptime
0          28140     40155  12:00:00  -77:00:00  2.439650e+06   3600.0
1          28141     40156  12:00:00  -64:00:00  2.439650e+06   3600.0
2          28142     40157  12:00:00  -51:00:00  2.439650e+06   3600.0
3          28143     40158  12:00:00  -38:00:00  2.439650e+06   3600.0
4          28144     40159  13:00:00  -77:00:00  2.439650e+06   3600.0
..           ...       ...       ...        ...           ...      ...
165        28307     40320  20:00:00  -38:00:00  2.439678e+06   3600.0
166        28308     40321  20:00:00  -77:00:00  2.439678e+06   3600.0
167        28310     40322  23:00:00  -51:00:00  2.439678e+06   3600.0
168        28311     40323  23:00:00  -38:00:00  2.439678e+06   3600.0
169        28312     40324  23:00:00  -77:00:00  2.439678e+06   3600.0

[170 rows x 6 columns]
     exposure_id  plate_id   ra_orig   dec_orig      jd_start  exptime
0       

In [20]:
# Initialize an empty DataFrame to store the combined results
combined_df = pd.DataFrame()

# Combine the query results into a single DataFrame
for result in query_results:
    df_result = result.to_table().to_pandas()
    combined_df = pd.concat([combined_df, df_result], ignore_index=True)

# Save the combined DataFrame as a CSV file named 'plate_tls.csv'
combined_df.to_csv('plate_dro_ss.csv', index=False)

In [21]:
import pandas as pd

# Read the CSV files into DataFrames
df1 = pd.read_csv('plate_dro.csv')
df2 = pd.read_csv('Dr._Remeis-Observatory,_Southern_Station.csv')

# Merge the DataFrames on 'plate_id'
merged_df = pd.merge(df2, df1[['plate_id', 'ra_orig', 'dec_orig']], on='plate_id')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('DRO_SS_plate_pos.csv', index=False)

print("Merged file saved as 'DRO_SS_plate_pos.csv'")

Merged file saved as 'DRO_SS_plate_pos.csv'


In [22]:
import pandas as pd
from datetime import datetime, timedelta

# Function to filter plates taken within a one-year period and have different filters
def filter_plates_within_year(df):
    filtered_plates = []
    for i in range(len(df)):
        for j in range(i+1, len(df)):
            date_i = datetime.strptime(df.iloc[i]['date_orig'], '%Y-%m-%d')
            date_j = datetime.strptime(df.iloc[j]['date_orig'], '%Y-%m-%d')
            delta = date_j - date_i
            if abs(delta.days) <= 124 and df.iloc[i]['filter'] != df.iloc[j]['filter']:
                filtered_plates.append(df.iloc[i])
                filtered_plates.append(df.iloc[j])
    return pd.DataFrame(filtered_plates)

# Read the CSV file
filename = "DRO_SS_plate_pos.csv"
# filename = "DRO_plate_pos.csv"
# filename = "ESO_plate_pos.csv"
# filename = "AOP_plate_pos.csv"
# filename = "HS_plate_pos.csv"
# filename = "TLS_plate_pos.csv"
# filename = "Vatican_Observatory.csv"
# filename = "Dr._Remeis-Observatory.csv"
df = pd.read_csv(filename)

# Sort by date_orig
df_sorted = df.sort_values(by='date_orig')

# Filter plates within one-year period with different filters
filtered_df = filter_plates_within_year(df_sorted)

# Save filtered DataFrame to CSV
filtered_df.to_csv("DRO_SS.csv", index=False)

In [24]:
import requests
import pyvo as vo
import json
import numpy as np
import pandas as pd

# Define the parameters
name = 'APPLAUSE'
url = 'https://www.plate-archive.org/tap'
token = 'Token 863d6c56ca291ffc99090f6b2cefb35f496ca8e4'

# Read the CSV file and select the column with the plate_id
df = pd.read_csv('Vatican_Observatory.csv')
plate_ids = df['plate_id'].tolist()

# Function to chunk the list into groups of specified size
def chunk_list(data_list, chunk_size):
    for i in range(0, len(data_list), chunk_size):
        yield data_list[i:i + chunk_size]

# Function to process each chunk
def process_chunk(chunk):
    # Format the plate_id list for the SQL query
    ids_str = ', '.join(f"'{id}'" for id in chunk)
    
    # Create the query string
    qstr = f"""
    SELECT exposure_id, plate_id, ra_orig, dec_orig, jd_start, exptime
    FROM applause_dr4.exposure
    WHERE plate_id IN ({ids_str})
    """
    
    # Setup the TAP service session
    tap_session = requests.Session()
    tap_session.headers['Authorization'] = token
    tap_service = vo.dal.TAPService(url, session=tap_session)
    
    # Submit the query
    lang = 'PostgreSQL'
    job = tap_service.submit_job(qstr, language=lang)
    job.run()
    
    # Wait for job completion
    job.wait(phases=["COMPLETED", "ERROR", "ABORTED"], timeout=30.)
    
    # Raise an error if the job failed
    job.raise_if_error()
    
    # Fetch results
    return job.fetch_result()

# Print pyvo version and TAP service name
print('\npyvo version %s \n' % vo.__version__)
print('TAP service %s \n' % name)

# Initialize the result list
query_results = []

# Process each chunk
for chunk in chunk_list(plate_ids, 170):
    results = process_chunk(chunk)
    query_results.append(results)

# Print the final query results as DataFrame
for result in query_results:
    df_result = result.to_table().to_pandas()
    print(df_result)


pyvo version 1.5.2 

TAP service APPLAUSE 

   exposure_id  plate_id   ra_orig   dec_orig      jd_start  exptime
0       139009     95879  19:54:00  +39:12:00  2.433908e+06   5040.0
1       139010     95880  19:54:00  +39:12:00  2.433909e+06    900.0
2       139041     95911  19:41:00  +37:54:00  2.433985e+06   1800.0
3       139044     95914  19:41:00  +37:54:00  2.433986e+06   7200.0
4       139050     95920  19:54:00  +39:12:00  2.433997e+06   1800.0
5       139051     95921  19:41:00  +37:54:00  2.433997e+06   1800.0
6       139075     95945  19:41:00  +37:09:00  2.434004e+06   1800.0
7       139076     95946  19:54:00  +39:12:00  2.434004e+06   1800.0


In [25]:
# Initialize an empty DataFrame to store the combined results
combined_df = pd.DataFrame()

# Combine the query results into a single DataFrame
for result in query_results:
    df_result = result.to_table().to_pandas()
    combined_df = pd.concat([combined_df, df_result], ignore_index=True)

# Save the combined DataFrame as a CSV file named 'plate_tls.csv'
combined_df.to_csv('plate_vo.csv', index=False)

In [26]:
import pandas as pd

# Read the CSV files into DataFrames
df1 = pd.read_csv('plate_vo.csv')
df2 = pd.read_csv('Vatican_Observatory.csv')

# Merge the DataFrames on 'plate_id'
merged_df = pd.merge(df2, df1[['plate_id', 'ra_orig', 'dec_orig']], on='plate_id')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('VO_plate_pos.csv', index=False)

print("Merged file saved as 'VO_plate_pos.csv'")

Merged file saved as 'VO_plate_pos.csv'


In [31]:
import pandas as pd
from datetime import datetime, timedelta

# Function to filter plates taken within a one-year period and have different filters
def filter_plates_within_year(df):
    filtered_plates = []
    for i in range(len(df)):
        for j in range(i+1, len(df)):
            date_i = datetime.strptime(df.iloc[i]['date_orig'], '%Y-%m-%d')
            date_j = datetime.strptime(df.iloc[j]['date_orig'], '%Y-%m-%d')
            delta = date_j - date_i
            if abs(delta.days) <= 124 and df.iloc[i]['filter'] != df.iloc[j]['filter']:
                filtered_plates.append(df.iloc[i])
                filtered_plates.append(df.iloc[j])
    return pd.DataFrame(filtered_plates)

# Read the CSV file
filename = "VO_plate_pos.csv"
# filename = "DRO_SS_plate_pos.csv"
# filename = "DRO_plate_pos.csv"
# filename = "ESO_plate_pos.csv"
# filename = "AOP_plate_pos.csv"
# filename = "HS_plate_pos.csv"
# filename = "TLS_plate_pos.csv"
# filename = "Vatican_Observatory.csv"
# filename = "Dr._Remeis-Observatory.csv"
df = pd.read_csv(filename)
# print(df)

# Sort by date_orig
df_sorted = df.sort_values(by='date_orig')
# print(df_sorted)

# Filter plates within one-year period with different filters
filtered_df = filter_plates_within_year(df_sorted)
print(filtered_df)

# Save filtered DataFrame to CSV
filtered_df.to_csv("VO.csv", index=False)

Empty DataFrame
Columns: []
Index: []
